In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.functions import isnan, count, when, col, desc, udf, col, sort_array, asc, avg
import datetime
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

In [0]:
#Creat Spark Session
spark=SparkSession.builder.appName("DW_BikeShare").getOrCreate()

In [0]:
#Create Filestore paths
payments_file_path="/FileStore/payments.csv"
rider_file_path="/FileStore/riders.csv"
stations_file_path="/FileStore/stations.csv"
trips_file_path="/FileStore/trips.csv"


In [0]:
#Create payment dataframe from csv file
df_pay=spark.read.csv(payments_file_path,header=False,inferSchema=True)

In [0]:
#Create rider dataframe from csv file
df_rider=spark.read.csv(rider_file_path,header=False,inferSchema=True)

In [0]:
#Create stations dataframe from csv file
df_stations=spark.read.csv(stations_file_path,header=False,inferSchema=True)

In [0]:
#Create trips dataframe from csv file
df_trips=spark.read.csv(trips_file_path,header=False,inferSchema=True)

In [0]:
#Add header record to the dataframe
df_riders=df_rider.toDF("rider_id","first","last","address","birthday","account_start_date","account_end_date","is_member")

In [0]:
df_riders.printSchema()

root
 |-- rider_id: integer (nullable = true)
 |-- first: string (nullable = true)
 |-- last: string (nullable = true)
 |-- address: string (nullable = true)
 |-- birthday: date (nullable = true)
 |-- account_start_date: date (nullable = true)
 |-- account_end_date: date (nullable = true)
 |-- is_member: boolean (nullable = true)



In [0]:
df_riders.dropDuplicates()

DataFrame[rider_id: int, first: string, last: string, address: string, birthday: date, account_start_date: date, account_end_date: date, is_member: boolean]

In [0]:
#Add header record to the dataframe
df_trip=df_trips.toDF("trip_id","rideable_type","started_at","ended_at","start_station_id","end_station_id","rider_id")

In [0]:
df_trip.printSchema()

root
 |-- trip_id: string (nullable = true)
 |-- rideable_type: string (nullable = true)
 |-- started_at: timestamp (nullable = true)
 |-- ended_at: timestamp (nullable = true)
 |-- start_station_id: string (nullable = true)
 |-- end_station_id: string (nullable = true)
 |-- rider_id: integer (nullable = true)



In [0]:
df_trip.dropDuplicates()

DataFrame[trip_id: string, rideable_type: string, started_at: timestamp, ended_at: timestamp, start_station_id: string, end_station_id: string, rider_id: int]

In [0]:
#Add header record to the dataframe
df_station=df_stations.toDF("station_id","name","latitude","longitude")

In [0]:
#Drop Duplicates
df_station.dropDuplicates()

DataFrame[station_id: string, name: string, latitude: double, longitude: double]

In [0]:
df_station.printSchema()

root
 |-- station_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)



In [0]:
#Add header record to the dataframe
df_payment=df_pay.toDF("id","date","amount","rider_id")

In [0]:
df_payment.show(5)

+---+----------+------+--------+
| id|      date|amount|rider_id|
+---+----------+------+--------+
|  1|2019-05-01|   9.0|    1000|
|  2|2019-06-01|   9.0|    1000|
|  3|2019-07-01|   9.0|    1000|
|  4|2019-08-01|   9.0|    1000|
|  5|2019-09-01|   9.0|    1000|
+---+----------+------+--------+
only showing top 5 rows



In [0]:
#df_payment.printSchema()
df_payment.dropDuplicates()

DataFrame[id: int, date: date, amount: double, rider_id: int]

In [0]:
#df_payment.count()
#1946607
#Create Delta file for Payment
df_payment.write.format("delta").mode("overwrite").save("/delta/bronze_payment")

In [0]:
df_payment.show(5)

+---+----------+------+--------+
| id|      date|amount|rider_id|
+---+----------+------+--------+
|  1|2019-05-01|   9.0|    1000|
|  2|2019-06-01|   9.0|    1000|
|  3|2019-07-01|   9.0|    1000|
|  4|2019-08-01|   9.0|    1000|
|  5|2019-09-01|   9.0|    1000|
+---+----------+------+--------+
only showing top 5 rows



In [0]:
df_p=spark.read.format("delta").load("/delta/bronze_payment")

In [0]:
df_p.head(5)

[Row(id=539256, date=datetime.date(2020, 8, 1), amount=9.0, rider_id=21826),
 Row(id=539257, date=datetime.date(2020, 9, 1), amount=9.0, rider_id=21826),
 Row(id=539258, date=datetime.date(2020, 10, 1), amount=9.0, rider_id=21826),
 Row(id=539259, date=datetime.date(2020, 11, 1), amount=9.0, rider_id=21826),
 Row(id=539260, date=datetime.date(2020, 12, 1), amount=9.0, rider_id=21826)]

In [0]:
##Create Delta table for Payment
spark.sql ("CREATE TABLE IF NOT EXISTS silver_payment USING DELTA LOCATION '/delta/bronze_payment' ")

DataFrame[]

In [0]:
#Create Delta table for rider
df_riders.write.format("delta").mode("overwrite").save("/delta/bronze_rider")
spark.sql ("CREATE TABLE IF NOT EXISTS silver_rider USING DELTA LOCATION '/delta/bronze_rider' ")

DataFrame[]

In [0]:
#Create Delta table for trips data
df_trip.write.format("delta").mode("overwrite").save("/delta/bronze_trip")
spark.sql ("CREATE TABLE IF NOT EXISTS silver_trip USING DELTA LOCATION '/delta/bronze_trip' ")

DataFrame[]

In [0]:
#Create Delta table for Stations data
df_station.write.format("delta").mode("overwrite").save("/delta/bronze_station")
spark.sql ("CREATE TABLE IF NOT EXISTS silver_station USING DELTA LOCATION '/delta/bronze_station' ")

DataFrame[]

In [0]:
#Create Payment Fact table 
spark.sql('''CREATE TABLE IF NOT EXISTS payment_fact USING DELTA AS 
          SELECT DISTINCT id AS payment_id, date,amount,rider_id FROM silver_payment''')

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
spark.sql("Select * from payment_fact limit 2").show()

+----------+----------+------+--------+
|payment_id|      date|amount|rider_id|
+----------+----------+------+--------+
|       165|2019-02-01| 15.47|    1007|
|       342|2019-09-01|   9.0|    1014|
+----------+----------+------+--------+



In [0]:
spark.sql('''CREATE TABLE IF NOT EXISTS dim_rider USING DELTA AS 
          SELECT DISTINCT * FROM silver_rider''')

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
spark.sql("select * from dim_rider  limit 2").show()

+--------+-------+---------+--------------------+----------+------------------+----------------+---------+
|rider_id|  first|     last|             address|  birthday|account_start_date|account_end_date|is_member|
+--------+-------+---------+--------------------+----------+------------------+----------------+---------+
|    1040|Matthew|    Watts|   323 Matthew Flats|1996-12-24|        2014-11-26|            NULL|     true|
|    1065|Rebecca|Mccormick|4912 Smith Alley ...|1983-07-18|        2017-07-13|            NULL|    false|
+--------+-------+---------+--------------------+----------+------------------+----------------+---------+



In [0]:
spark.sql('''CREATE TABLE IF NOT EXISTS dim_station USING DELTA AS 
          SELECT DISTINCT  *  FROM silver_station''')

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
spark.sql("select * from dim_station limit 2").show()

+------------+--------------------+-----------------+------------------+
|  station_id|                name|         latitude|         longitude|
+------------+--------------------+-----------------+------------------+
|       15550|Canal St & Taylor St|        41.870257|-87.63947399999999|
|TA1307000150|Pine Grove Ave & ...|41.94947274088333|-87.64645278453827|
+------------+--------------------+-----------------+------------------+



In [0]:
#Create Dim Ridetype table
spark.sql ('''
           CREATE TABLE IF NOT EXISTS dim_ride_type
USING DELTA
AS
SELECT
  ROW_NUMBER() OVER (ORDER BY rideable_type) AS ride_type_id,
  rideable_type
FROM
  silver_trip
GROUP BY
  rideable_type
  '''
           
           )

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
spark.sql ("select * from dim_ride_type").show()

+------------+-------------+
|ride_type_id|rideable_type|
+------------+-------------+
|           1| classic_bike|
|           2|  docked_bike|
|           3|electric_bike|
+------------+-------------+



In [0]:
spark.sql ("select * from payment_fact where rider_id=34062").show()

+----------+----------+------+--------+
|payment_id|      date|amount|rider_id|
+----------+----------+------+--------+
|    856454|2021-01-01|   9.0|   34062|
|    856453|2020-12-01|   9.0|   34062|
+----------+----------+------+--------+



In [0]:
spark.sql ("SELECT DISTINCT id AS payment_id, date,amount,rider_id FROM silver_payment limit 5").show()

+----------+----------+------+--------+
|payment_id|      date|amount|rider_id|
+----------+----------+------+--------+
|       165|2019-02-01| 15.47|    1007|
|       342|2019-09-01|   9.0|    1014|
|       398|2019-10-01|   9.0|    1015|
|       543|2020-04-01|   9.0|    1022|
|       554|2021-03-01|   9.0|    1022|
+----------+----------+------+--------+



In [0]:
# Trip Counts by rider in descending order.
spark.sql ('''select r.rider_id,count(distinct trip_id)  as Trip_Counts from silver_trip t join silver_rider r on t.rider_id=r.rider_id 
           group by r.rider_id
           Order by count(distinct trip_id) desc
            ''').show()

+--------+-----------+
|rider_id|Trip_Counts|
+--------+-----------+
|   53044|       1636|
|   37388|       1584|
|   21973|       1537|
|   33748|       1529|
|   14363|       1480|
|   61581|       1433|
|    5211|       1422|
|   66814|       1398|
|   67878|       1382|
|    4193|       1367|
|   71748|       1366|
|   19330|       1361|
|   39417|       1355|
|   52113|       1346|
|   43849|       1346|
|   25890|       1345|
|   50994|       1343|
|   55633|       1339|
|   55955|       1335|
|    6048|       1333|
+--------+-----------+
only showing top 20 rows



In [0]:
spark.sql ('''select r.rider_id, t.trip_id,FLOOR(DATEDIFF(t.started_at, r.birthday) / 365.25) AS age 
           ,sum(amount) as trip_amount
           from silver_trip t join silver_rider r on t.rider_id=r.rider_id 
join silver_payment p on p.rider_id=r.rider_id           
Group By r.rider_id, t.trip_id,FLOOR(DATEDIFF(t.started_at, r.birthday) / 365.25)
Order by rider_id
           ''').show()

+--------+----------------+---+-----------+
|rider_id|         trip_id|age|trip_amount|
+--------+----------------+---+-----------+
|    1000|794BFB4727EFB58B| 32|      306.0|
|    1000|D3BE89A24107DA70| 32|      306.0|
|    1000|E2C779C18C7152A9| 32|      306.0|
|    1000|632A33F2C25D5990| 32|      306.0|
|    1000|957978AA49C230A5| 32|      306.0|
|    1000|24FB5B82406EB7B8| 32|      306.0|
|    1000|DA619FAE456C953E| 32|      306.0|
|    1000|3617865036B0468A| 32|      306.0|
|    1000|F0C6DD1F7754C733| 32|      306.0|
|    1000|B7CFA527619455F5| 32|      306.0|
|    1000|B37B922E4A2087E9| 32|      306.0|
|    1000|D8CDC7E28EF69BB7| 32|      306.0|
|    1000|A44F47CF28EE6F6B| 32|      306.0|
|    1000|2BFCC9CA079188B2| 32|      306.0|
|    1000|4EBB0B75CACB886C| 32|      306.0|
|    1000|0FC0422D68F59E02| 32|      306.0|
|    1000|9576415E314CD525| 32|      306.0|
|    1003|5D1E06F75542A21C| 22|     411.07|
|    1003|E69CF13359BB0177| 22|     411.07|
|    1003|5FB921A23BCC2A09| 22| 

In [0]:
spark.sql ("select rider_id,count(distinct payment_id) from payment_fact group by rider_id having count(distinct payment_id) > 1").show()

+--------+--------------------------+
|rider_id|count(DISTINCT payment_id)|
+--------+--------------------------+
|   18979|                        49|
|    9427|                        21|
|   13289|                        68|
|   18866|                        52|
|    3175|                        11|
|    3749|                         7|
|   38723|                        45|
|   35361|                        38|
|   33722|                         6|
|   37307|                        36|
|   26706|                        11|
|   41890|                        88|
|   44358|                        73|
|   46943|                        24|
|   53963|                        49|
|   43103|                        96|
|    7240|                        23|
|   41751|                        53|
|   28836|                        68|
|   55283|                        20|
+--------+--------------------------+
only showing top 20 rows



In [0]:
spark.sql ('''
           CREATE TABLE IF NOT EXISTS trip_fact USING DELTA AS 

           select t.trip_id ,t.rider_id,t.start_station_id,t.end_station_id,rt.ride_type_id,started_at,ended_at,
           round(datediff (second, t.started_at, t.ended_at) / 60)  AS trip_duration,
            floor (months_between (t.started_at, r.birthday) / 12) as rider_age,r.is_member
from silver_trip t
join dim_rider r on t.rider_id=r.rider_id
join dim_station s on s.station_id=t.start_station_id
join dim_station se on se.station_id=t.end_station_id
join dim_ride_type rt on rt.rideable_type=t.rideable_type

'''
           
           
           )

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
spark.sql ("select * from trip_fact limit 2").show()

+----------------+--------+----------------+--------------+------------+-------------------+-------------------+-------------+---------+---------+
|         trip_id|rider_id|start_station_id|end_station_id|ride_type_id|         started_at|           ended_at|trip_duration|rider_age|is_member|
+----------------+--------+----------------+--------------+------------+-------------------+-------------------+-------------+---------+---------+
|7E1E50AC37E2DAD3|    2644|    TA1309000007|         13089|           1|2021-08-14 14:01:36|2021-08-14 14:34:49|         33.0|       45|     true|
|ADFF32195521E952|   37747|           13288|  TA1308000031|           1|2021-08-29 16:16:36|2021-08-29 16:24:43|          8.0|       19|     true|
+----------------+--------+----------------+--------------+------------+-------------------+-------------------+-------------+---------+---------+



In [0]:
spark.sql ("select min(started_at) as min_date,max(ended_at) as max_Date from trip_fact limit 2").show()

+-------------------+-------------------+
|           min_date|           max_Date|
+-------------------+-------------------+
|2021-02-01 01:07:04|2022-02-01 00:12:04|
+-------------------+-------------------+



In [0]:
import pyspark.sql.functions as F 
min_date=spark.sql("select date(min(started_at)) from trip_fact").collect()[0][0]
#print(min_date)
max_date=spark.sql("select date(max(started_at)) from trip_fact").collect()[0][0]
#print(max_date)
date_dim = spark.range(0, (max_date - min_date).days + 1) \
    .withColumn("date", F.date_add(F.lit(min_date), F.col("id").cast("integer"))) \
    .withColumn("day_of_week", F.dayofweek("date").cast("integer")) \
    .withColumn("day_of_month", F.dayofmonth("date").cast("integer")) \
    .withColumn("day_of_year", F.dayofyear("date").cast("integer")) \
    .withColumn("week_of_year", F.weekofyear("date").cast("integer")) \
    .withColumn("month", F.month("date").cast("integer")) \
    .withColumn("month_name", F.monthname("date")) \
    .withColumn("quarter", F.quarter("date").cast("integer")) \
    .withColumn("year", F.year("date").cast("integer"))

In [0]:
date_dim.show(5)

+---+----------+-----------+------------+-----------+------------+-----+----------+-------+----+
| id|      date|day_of_week|day_of_month|day_of_year|week_of_year|month|month_name|quarter|year|
+---+----------+-----------+------------+-----------+------------+-----+----------+-------+----+
|  0|2021-02-01|          2|           1|         32|           5|    2|       Feb|      1|2021|
|  1|2021-02-02|          3|           2|         33|           5|    2|       Feb|      1|2021|
|  2|2021-02-03|          4|           3|         34|           5|    2|       Feb|      1|2021|
|  3|2021-02-04|          5|           4|         35|           5|    2|       Feb|      1|2021|
|  4|2021-02-05|          6|           5|         36|           5|    2|       Feb|      1|2021|
+---+----------+-----------+------------+-----------+------------+-----+----------+-------+----+
only showing top 5 rows



In [0]:
date_dim.write.format("delta").mode("overwrite").saveAsTable("date_dim")

  

In [0]:
spark.sql("select * from date_dim limit 5").show()

+---+----------+-----------+------------+-----------+------------+-----+----------+-------+----+
| id|      date|day_of_week|day_of_month|day_of_year|week_of_year|month|month_name|quarter|year|
+---+----------+-----------+------------+-----------+------------+-----+----------+-------+----+
|273|2021-11-01|          2|           1|        305|          44|   11|       Nov|      4|2021|
|274|2021-11-02|          3|           2|        306|          44|   11|       Nov|      4|2021|
|275|2021-11-03|          4|           3|        307|          44|   11|       Nov|      4|2021|
|276|2021-11-04|          5|           4|        308|          44|   11|       Nov|      4|2021|
|277|2021-11-05|          6|           5|        309|          44|   11|       Nov|      4|2021|
+---+----------+-----------+------------+-----------+------------+-----+----------+-------+----+



Data Anaylysis  how much time is spent per ride Based on the following Factors:
- Based on date and time factors such as day of week and time of day
- Based on which station is the starting and / or ending station
- Based on age of the rider at time of the ride
- Based on whether the rider is a member or a casual rider

In [0]:
# Analyze how much time is spent per ride -->  Based on date and time factors such as day of week and time of day

rider_stats_by_day_hour=spark.sql (  '''SELECT rider_id, DAY(started_at) as DAY, extract(HOUR FROM started_at) as HOUR, count(*) as num_rides, AVG( DATEDIFF (second, started_at, ended_at)) * 60 as minutes_spent
    FROM trip_fact
    GROUP by rider_id, DAY(started_at),extract(HOUR FROM started_at)
    Order By rider_id,DAY(started_at),extract(HOUR FROM started_at)'''
    )
rider_stats_by_day_hour.createOrReplaceTempView("rider_stats_by_day_hour")
spark.sql ("Select * from rider_stats_amount limit 5").show()

+--------+----+-------+-----+------------+----------+
|rider_id|year|quarter|month|Total_Amount|Avg_Amount|
+--------+----+-------+-----+------------+----------+
|    1000|2021|      1|    2|         9.0|       9.0|
|    1000|2021|      1|    3|         9.0|       9.0|
|    1000|2021|      2|    4|         9.0|       9.0|
|    1000|2021|      2|    5|         9.0|       9.0|
|    1000|2021|      2|    6|         9.0|       9.0|
+--------+----+-------+-----+------------+----------+



In [0]:
# How Much Time Spent based on which station is the starting and / or ending station
trp_stats_by_stations=spark.sql("select start_station_id,end_station_id,count(distinct trip_id) as trip_count,sum(trip_duration) as timespent,avg(trip_duration) as avg_time from trip_fact group by start_station_id,end_station_id")
trp_stats_by_stations.createOrReplaceTempView("trp_stats_by_stations")
spark.sql("select * from trp_stats_by_stations limit 2").show()

+----------------+--------------+----------+---------+------------------+
|start_station_id|end_station_id|trip_count|timespent|          avg_time|
+----------------+--------------+----------+---------+------------------+
|    TA1307000121|  TA1307000044|        51|    520.0|10.196078431372548|
|    TA1308000022|  TA1308000022|      1294|  51259.0|39.612828438948995|
+----------------+--------------+----------+---------+------------------+



In [0]:
# How Much Time Spent per rider based on which station is the starting and / or ending station

trp_stats_by_stations=spark.sql("select rider_id,start_station_id,end_station_id,count(distinct trip_id) as trip_count,sum(trip_duration) as timespent,avg(trip_duration) as avg_time from trip_fact group by rider_id,start_station_id,end_station_id")
trp_stats_by_stations.createOrReplaceTempView("trp_stats_by_rider_stations")
spark.sql("select * from trp_stats_by_rider_stations order by rider_id limit 5 ").show()

+--------+----------------+--------------+----------+---------+--------+
|rider_id|start_station_id|end_station_id|trip_count|timespent|avg_time|
+--------+----------------+--------------+----------+---------+--------+
|    1000|    KA1504000126|  TA1309000014|         2|     35.0|    17.5|
|    1000|    KA1504000126|  TA1309000030|         1|     43.0|    43.0|
|    1000|    KA1504000126|  KA1504000096|         1|      9.0|     9.0|
|    1000|    KA1504000126|  KA1504000126|         1|     45.0|    45.0|
|    1000|    KA1504000126|         13379|         1|     29.0|    29.0|
+--------+----------------+--------------+----------+---------+--------+



In [0]:
#Analyze how much time is spent per ride
#Based on age of the rider at time of the ride
rider_stats_by_age=spark.sql("select rider_age,count(trip_id) as trp_count,avg(trip_duration) as avg_timespent from trip_fact group by rider_age")

rider_stats_by_age.createOrReplaceTempView("rider_avg_stats_by_age")
spark.sql("select * from rider_avg_stats_by_age ORder By rider_age desc limit 2").show()


+---------+---------+------------------+
|rider_age|trp_count|     avg_timespent|
+---------+---------+------------------+
|       75|       45|              51.4|
|       74|     1768|19.059954751131222|
+---------+---------+------------------+



In [0]:
#Based on whether the rider is a member or a casual rider
rider_stats_by_member_type=spark.sql("select  CASE WHEN is_member=true then 'member' else 'casualrider' end as Member_type,count(trip_id) as trp_count,round(sum(trip_duration)/60 ) as total_timespent_hrs from trip_fact group by  CASE WHEN is_member=true then 'member' else 'casualrider' end ")
rider_stats_by_member_type.createOrReplaceTempView("rider_stats_by_member_type")
spark.sql("select * from rider_stats_by_member_type limit 2").show()

+-----------+---------+-------------------+
|Member_type|trp_count|total_timespent_hrs|
+-----------+---------+-------------------+
|casualrider|   918615|           326582.0|
|     member|  3666306|          1338940.0|
+-----------+---------+-------------------+



In [0]:

# aggregate count of rides and duration by rider, by month
rider_stats_month_year = spark.sql(
    """
    SELECT  MONTH(started_at) as month, YEAR(started_at) as year, count(*) as num_rides, AVG( DATEDIFF (second, started_at, ended_at)) * 60 as minutes_spent
    FROM trip_fact
    GROUP by  MONTH(started_at), YEAR(started_at);
    """
)
rider_stats_month_year.createOrReplaceTempView("rider_stats_month_year")
spark.sql("select * from rider_stats_month_year limit 2").show()

+-----+----+---------+-----------------+
|month|year|num_rides|    minutes_spent|
+-----+----+---------+-----------------+
|    7|2021|   692321|87092.90999406345|
|    8|2021|   674409|76150.34105416742|
+-----+----+---------+-----------------+



Analyze how much money is spent
- Per month, quarter, year
- Per member, based on the age of the rider at account start

In [0]:
#how much money is spent Per month, quarter, year Per Rider

rider_stats_amount=spark.sql ('''
           select r.rider_id,dat.year ,dat.quarter , dat.month ,sum(amount) as Total_Amount, Avg(amount) Avg_Amount
           From
           dim_rider r join payment_fact p on r.rider_Id=p.rider_Id
           join date_dim dat on dat.date=p.date
           
           Group By r.rider_id,dat.year ,dat.quarter , dat.month
           Order By r.rider_id,dat.year ,dat.quarter , dat.month
           ''')
rider_stats_amount.createOrReplaceTempView("rider_stats_amount")
spark.sql("select * from rider_stats_amount limit 5").show()

+--------+----+-------+-----+------------+----------+
|rider_id|year|quarter|month|Total_Amount|Avg_Amount|
+--------+----+-------+-----+------------+----------+
|    1000|2021|      1|    2|         9.0|       9.0|
|    1000|2021|      1|    3|         9.0|       9.0|
|    1000|2021|      2|    4|         9.0|       9.0|
|    1000|2021|      2|    5|         9.0|       9.0|
|    1000|2021|      2|    6|         9.0|       9.0|
+--------+----+-------+-----+------------+----------+



In [0]:
money_spent=spark.sql ('''
           select dat.year ,dat.quarter , dat.month ,sum(amount) as Total_Amount, Avg(amount) Avg_Amount
           From
           dim_rider r join payment_fact p on r.rider_Id=p.rider_Id
           join date_dim dat on dat.date=p.date
           
           Group By dat.year ,dat.quarter , dat.month
           Order By dat.year ,dat.quarter , dat.month
           ''')
money_spent.createOrReplaceTempView("money_spent")
spark.sql("select * from money_spent limit 5").show()

+----+-------+-----+------------------+-----------------+
|year|quarter|month|      Total_Amount|       Avg_Amount|
+----+-------+-----+------------------+-----------------+
|2021|      1|    2|447430.36999999965|10.01029979640691|
|2021|      1|    3|458784.17000000004| 9.99203245126865|
|2021|      2|    4| 471751.6599999996|9.983528241593119|
|2021|      2|    5|485719.24999999907|9.992783961157839|
|2021|      2|    6| 500206.6499999999|9.996136091127097|
+----+-------+-----+------------------+-----------------+



In [0]:
#Per member, based on the age of the rider at account start

rider_stats_by_age=spark.sql('''select floor (months_between (account_start_Date, birthday) / 12) as rider_age,sum(p.amount) as Total_Amt   From
           dim_rider r join payment_fact p on r.rider_Id=p.rider_Id
           where is_member=true
          Group By floor (months_between (account_start_Date, birthday) / 12)
                     ''')
rider_stats_by_age.createOrReplaceTempView("rider_stats_by_age")
spark.sql("select * from rider_stats_by_age limit 4").show()

+---------+---------+
|rider_age|Total_Amt|
+---------+---------+
|       29| 438219.0|
|       26| 470439.0|
|       65|   8469.0|
|       19| 456030.0|
+---------+---------+



In [0]:
# how many minutes the rider spends on a bike per month
spark.sql ('''
           select rider_id,MONTH(started_at) as Month,count(distinct trip_id) as trip_count, sum(trip_duration) as total_trip_mns 
           ,AVG(COUNT(DISTINCT trip_id)) OVER (PARTITION BY rider_id, MONTH(started_at)) AS avg_monthly_rides
             from trip_fact 
           Group by rider_id,MONTH(started_at)
           limit 4
           ''').show()

+--------+-----+----------+--------------+-----------------+
|rider_id|Month|trip_count|total_trip_mns|avg_monthly_rides|
+--------+-----+----------+--------------+-----------------+
|    1000|    4|         2|          34.0|              2.0|
|    1000|   11|         3|          58.0|              3.0|
|    1003|    6|        18|         296.0|             18.0|
|    1003|   10|        16|         190.0|             16.0|
+--------+-----+----------+--------------+-----------------+



In [0]:
# how many minutes the rider spends on a bike per month
spark.sql ('''
           select rider_id,MONTH(started_at) as Month,count(distinct trip_id) as trip_count, sum(trip_duration) as total_trip_mns 
           ,AVG(COUNT(DISTINCT trip_id)) OVER (PARTITION BY rider_id, MONTH(started_at)) AS avg_monthly_rides
             from trip_fact t join Dim_re
           Group by rider_id,MONTH(started_at)
           limit 4
           ''').show()